In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
BASE_DIR = '..'
NIGHTLIGHTS_DIR = os.path.join(BASE_DIR, 'data/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif')
COUNTRY = 'timor_2016'

COUNTRY_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY)
DHS_DIR = os.path.join(COUNTRY_DIR, 'DHS')

In [13]:
dhs_file = DHS_DIR + '/TLHR71DT/TLHR71FL.DTA'
dhs_dict_file = DHS_DIR + '/TLHR71DT/TLHR71FL.DO'
dhs_gps = DHS_DIR + '/TLGE71FL/TL_DHS_GPS.dta'

In [14]:
def get_dhs_dict(dhs_dict_file):
    dhs_dict = dict()
    with open(dhs_dict_file, 'r', errors='replace') as file:
        line = file.readline()
        while line:
            line = file.readline()
            if 'label variable' in line:
                code = line.split()[2]
                colname = ' '.join([x.strip('"') for x in line.split()[3:]])
                dhs_dict[code] = colname
    return dhs_dict

In [15]:
# Load DHS data

In [16]:
dhs = pd.read_stata(dhs_file, convert_categoricals=False)
dhs_dict = get_dhs_dict(dhs_dict_file)
dhs = dhs.rename(columns=dhs_dict).dropna(axis=1)
print('Data Dimensions: {}'.format(dhs.shape))

Data Dimensions: (11502, 198)


In [17]:
dhs.head()

,Case Identification,Country code and phase,Cluster number,Household number,Respondent's line number (answering Household questionnaire),Ultimate area unit,Household sample weight (6 decimals),Month of interview,Year of interview,Date of interview (CMC),...,NA - Bar code for blood smear sample,Index to household schedule,Wear glasses or contact lenses,Have difficulty seeing,Have difficulty hearing,Have difficulty communicating using usual language,Have difficulty remembering or concentrating,Have difficulty walking or climbing steps,Have difficulty washing all over or dressing,Highest degree of difficulty for any of the impairments
0,1 1,TL7,1,1,1,1,478354,9,2016,1401,...,,1,0,1,1,1,1,1,1,1
1,1 2,TL7,1,2,2,1,478354,9,2016,1401,...,,1,1,2,1,1,1,1,1,2
2,1 3,TL7,1,3,1,1,478354,9,2016,1401,...,,1,1,1,1,1,1,1,1,1
3,1 4,TL7,1,4,4,1,478354,9,2016,1401,...,,1,1,1,2,1,1,1,1,2
4,1 5,TL7,1,5,3,1,478354,9,2016,1401,...,,1,1,1,1,1,1,1,1,1


In [ ]:
# Aggregate columns to clusters

In [18]:
data = dhs[[
    'Cluster number',
    'Wealth index factor score combined (5 decimals)',
    'Education completed in single years',
    'Has electricity'
]].groupby('Cluster number').mean()

data['Time to get to water source (minutes)'] = dhs[[
    'Cluster number',
    'Time to get to water source (minutes)'
]].replace(996, 0).groupby('Cluster number').median()

data.columns = [[
    'Wealth Index',
    'Education completed (years)',
    'Access to electricity',
    'Access to water (minutes)'
]]

print('Data Dimensions: {}'.format(data.shape))
data.head(5)

Data Dimensions: (455, 4)


,Wealth Index,Education completed (years),Access to electricity,Access to water (minutes)
Cluster number,,,,
1,32166.600000,6.280000,0.720000,0.0
2,-34063.923077,1.615385,0.961538,17.5
3,39230.590909,6.318182,0.954545,0.0
4,-82140.227273,1.181818,0.045455,60.0
5,-56203.423077,2.576923,0.384615,22.5


In [19]:
# Load GPS data

In [20]:
data_gps = pd.read_stata(dhs_gps, convert_categoricals=False)
data_gps.head()

,dhsclust,urban_rura,latnum,longnum
0,1,R,-8.712016,125.567383
1,2,R,-8.730226,125.590218
2,3,R,-8.741340,125.556396
3,4,R,-8.811291,125.535164
4,5,R,-8.791590,125.473221


In [22]:
#merge cluster data and gps data
data_combined = pd.merge(data,data_gps,left_on='Cluster number',right_on='dhsclust')
data_combined.head()

,"(Wealth Index,)","(Education completed (years),)","(Access to electricity,)","(Access to water (minutes),)",dhsclust,urban_rura,latnum,longnum
0,32166.600000,6.280000,0.720000,0.0,1,R,-8.712016,125.567383
1,-34063.923077,1.615385,0.961538,17.5,2,R,-8.730226,125.590218
2,39230.590909,6.318182,0.954545,0.0,3,R,-8.741340,125.556396
3,-82140.227273,1.181818,0.045455,60.0,4,R,-8.811291,125.535164
4,-56203.423077,2.576923,0.384615,22.5,5,R,-8.791590,125.473221


In [25]:
data_combined.columns =['wealth','education','electricity','water','cluster','urban','lat','lon']
data_combined.head()

,wealth,education,electricity,water,cluster,urban,lat,lon
0,32166.600000,6.280000,0.720000,0.0,1,R,-8.712016,125.567383
1,-34063.923077,1.615385,0.961538,17.5,2,R,-8.730226,125.590218
2,39230.590909,6.318182,0.954545,0.0,3,R,-8.741340,125.556396
3,-82140.227273,1.181818,0.045455,60.0,4,R,-8.811291,125.535164
4,-56203.423077,2.576923,0.384615,22.5,5,R,-8.791590,125.473221


In [23]:
# Merge nightlight data

In [28]:
import geoio
img = geoio.GeoImage(NIGHTLIGHTS_DIR)
#begining point of nighttime light data
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [29]:
import geoio
img = geoio.GeoImage(NIGHTLIGHTS_DIR)
# pass lon then lat
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)

In [30]:
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [31]:
im_array = np.squeeze(img.get_data())
im_array.shape

(16801, 43201)

In [32]:
im_array[int(yPixel),int(xPixel)] # this is the nightlight value at the given coordinate

0

In [33]:
import math

def create_space(lat, lon):
    # these are pulled from the paper to make the 10km^2 area
    return lat - (180/math.pi)*(5000/6378137), lon - (180/math.pi)*(5000/6378137)/math.cos(lat), \
            lat + (180/math.pi)*(5000/6378137), lon + (180/math.pi)*(5000/6378137)/math.cos(lat)

In [36]:
cluster_nightlights = []
for i,r in data_combined.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.lat, r.lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    cluster_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [37]:
data_combined['nightlights'] = cluster_nightlights

In [38]:
data_combined.head()

,wealth,education,electricity,water,cluster,urban,lat,lon,nightlights
0,32166.600000,6.280000,0.720000,0.0,1,R,-8.712016,125.567383,0.928571
1,-34063.923077,1.615385,0.961538,17.5,2,R,-8.730226,125.590218,1.201299
2,39230.590909,6.318182,0.954545,0.0,3,R,-8.741340,125.556396,1.021429
3,-82140.227273,1.181818,0.045455,60.0,4,R,-8.811291,125.535164,0.265734
4,-56203.423077,2.576923,0.384615,22.5,5,R,-8.791590,125.473221,0.076923


In [39]:
os.makedirs(os.path.join(COUNTRY_DIR, 'processed'), exist_ok=True)

In [41]:
data_combined.to_csv(os.path.join(COUNTRY_DIR, 'processed/clusters.csv'), index=False)